# Face recognition

In [24]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import Normalizer

## Load data

In [25]:
def load_data(pref, path):
    
    data = sio.loadmat(path)
    X = data[pref + "_img_sample"][:,0]
    label= data[pref + "_img_sample"][:,2]
    
    Y = []
    for l in label:
        Y.append(l[0][0])

#     encoder = LabelEncoder()
#     encoder.fit(label)
#     encoded_Y = encoder.transform(label)
#     Y = np_utils.to_categorical(encoded_Y)
    
    return X, Y

X_train, Y_train = load_data('tr', '../data/face_recognition/face_recognition_data_tr.mat')
X_val, Y_val = load_data('va', '../data/face_recognition/face_recognition_data_va.mat')

print("Elements for training: {:d}".format(X_train.shape[0]))
print("Elements for validation: {:d}".format(X_val.shape[0]))

Elements for training: 368
Elements for validation: 111


## Extract features

In [26]:
# from https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/2b62a1c3-e151-4a0e-b6b6-e424fa46ceab/lessons/fd66c083-4ccb-4fe3-bda1-c29db76f50a0/concepts/d479f43a-7bbb-4de7-9452-f6b991ece599
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                                  visualise=True, feature_vector=False)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=False, feature_vector=feature_vec)
        return features

def extract_features(image, orient=9, pix_per_cell=8, cell_per_block=2):
    hog_feats = get_hog_features(
            image, 
            orient, 
            pix_per_cell, 
            cell_per_block, 
            vis=False, 
            feature_vec=True
        )

    return hog_feats

def extract_features_lbp(image, num_points=12, radius=4, eps=1e-7):
    lbp = local_binary_pattern(image, num_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(),
                          bins=np.arange(0, num_points + 3),
                          range=(0, num_points + 2))
    # normalization
    hist = hist.astype(float)
    hist /= (hist.sum() + eps)
    
    return hist

def extract_features_all(X):

    X_hog = []
    X_lbp = []
    for el in X:
        h, l = extract_features(el), extract_features_lbp(el)
        X_hog.append(h)
        X_lbp.append(l)
        
    X_hog = np.array(X_hog)
    X_lbp = np.array(X_lbp)
    
    norm = Normalizer()
    
    norm.fit(X_hog)
    X_hog = norm.transform(X_hog)
    
    norm.fit(X_lbp)
    X_lbp = norm.transform(X_lbp)
    
    return np.hstack((X_hog, X_lbp))

In [27]:
Xfeat_train = extract_features_all(X_train)
Xfeat_val = extract_features_all(X_val)

/mnt/linux_shared/shared/anaconda3/envs/icv/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [28]:
print("Number of features: {:d}".format(Xfeat_train.shape[1]))

Number of features: 1778


## Train model

In [29]:
from sklearn.svm import SVC

model = SVC(kernel="linear", C=100.0)
model.fit(Xfeat_train, Y_train)
model.score(Xfeat_val, Y_val)
#model.predict(Xfeat_val)
# print(Y_val)

0.60360360360360366